# Batch inference with Bedrock

This notebook aims to provide an example on how to run batch inference with Bedrock. 

With batch inference, you can process a larger number of prompts in a more efficient way. Also, according to [pricing](https://aws.amazon.com/bedrock/pricing/), it is ~50% cheaper than doing inference with single requests.

Note: Not all the models support batch inference. Check [here](https://docs.aws.amazon.com/bedrock/latest/userguide/batch-inference-supported.html) the ones that do.

Note 2: SLA for Bedrock Batch Inference Jobs to start is ~24h. 

## Create the dataset

First, let's create some sample data for demonstration purposes:

In [1]:
from ml_utils.llm.input_builder import Message

SYSTEM_PROMPT = "Please respond only with emoji."
messages = [
    [Message(role="user", content="Hello there! How are you doing today?")],
    [Message(role="user", content="My cat is so cute!")],
    [Message(role="user", content="I'm programming today")],
] * 100  # there is a requirement in Bedrock for having min 100 input samples

llm_inputs = iter(messages)

## Run inference batch job

The way batch inference works with Bedrock is as follows:
- You create (or already have) a dataset with your prompts in an S3 bucket.
- You run the batch inference job. 
- The results are written in an S3 bucket. Then, if needed, you can download them.

We have abstractions to run all these operations easily.

First, let's define our BedrockBatchInference instance:

In [2]:
from pathlib import Path

from ml_utils.aws.bedrock.bedrock_batch_inference import BedrockBatchInference
from ml_utils.llm.input_builder import ClaudeInputBuilder

bedrock_batch_inference = BedrockBatchInference(
    llm=ClaudeInputBuilder(),
    out_path=Path("outputs"),
    in_uri="s3://ml-rd-bedrock-datasets/examples.jsonl",  # the input data will be written to this file
    out_uri="s3://ml-rd-bedrock-inference-outputs/",  # the output data will be written to this file
)

And now, let's run the batch job with some inputs:

In [3]:
bedrock_batch_inference.run(
    system=SYSTEM_PROMPT,
    inputs=llm_inputs,
    modelid="anthropic.claude-3-sonnet-20240229-v1:0",
    detached=True,  # not waiting for the job to finish
)

2025-02-19 12:36:31.112 | INFO     | ml_utils.aws.s3:upload_file_to_s3:24 - examples.jsonl already exists in s3://ml-rd-bedrock-datasets
2025-02-19 12:36:32.060 | INFO     | ml_utils.aws.s3:upload_file_to_s3:27 - Overwritten /var/folders/qv/xp56l9w94j7bwwvff2yf03680000gn/T/tmpavklcszx to s3://ml-rd-bedrock-datasets/examples.jsonl
2025-02-19 12:36:33.663 | INFO     | ml_utils.aws.bedrock.bedrock_batch_inference:_run:63 - Successfully launched job with ARN: arn:aws:bedrock:us-east-1:879381254630:model-invocation-job/wh68xt43r2ff


Now you can go to the Bedrock UI and check your job under Inference and Assessment > Batch inference.

Since we have run the job in detach mode, the execution will not wait until it completes. However, if you set detached=True when running the job, the execution will automatically wait and download the results to the provided ouput path (in our case, outputs/). Of course, this step is not recommended if the dataset is huge, as downloading the results might take lots of time and storage.

## More resources

- [Example from AWS](https://github.com/aws-samples/amazon-bedrock-samples/blob/main/introduction-to-bedrock/batch_api/batch-inference-transcript-summarization.ipynb)